### step 2

In [10]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [11]:
seed = 7
np.random.seed(seed)

In [24]:
dataframe = pd.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()

X = dataset[:,0:60].astype(float)

Y = dataset[:,60]
Y=le.fit_transform(dataset[:,60])



from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, shuffle= True)

one_hot_train_labels=to_categorical(y_train)
one_hot_test_labels=to_categorical(y_test)

x_val=x_train[:25]
partial_x_train=x_train[25:]

y_val=one_hot_train_labels[:25]
partial_y_train=one_hot_train_labels[25:]

In [13]:
def create_baseline():
    model=Sequential()
    model.add(Dense(32,activation='relu',input_shape=(60,)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    #model.fit(partial_x_train,partial_y_train,epochs=5,validation_data=(x_val,y_val))
    return model

In [5]:
estimator=KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
results=cross_val_score(estimator,X,Y,cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Results: 80.71% (5.02%)


## step 3 

In [8]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 85.11% (8.16%)


### step 4.1

In [14]:
def create_smaller():
    model=Sequential()
    model.add(Dense(30,activation='relu',input_shape=(60,)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    #model.fit(partial_x_train,partial_y_train,epochs=5,validation_data=(x_val,y_val))
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 84.59% (2.99%)


### step 4.2 and 5

In [ ]:
def create_larger():
    model=Sequential()
    model.add(Dense(30,activation='relu',input_shape=(60,)))
    model.add(Dense(30,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    #model.fit(partial_x_train,partial_y_train,epochs=5,validation_data=(x_val,y_val))
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Epoch 1/100
186/186 [==============================] - 6s 31ms/step - loss: 0.6452 - acc: 0.6344
Epoch 2/100
186/186 [==============================] - 0s 782us/step - loss: 0.5186 - acc: 0.7634
Epoch 3/100
186/186 [==============================] - 0s 946us/step - loss: 0.4345 - acc: 0.8065
Epoch 4/100
186/186 [==============================] - 0s 731us/step - loss: 0.3674 - acc: 0.8548
Epoch 5/100
186/186 [==============================] - 0s 688us/step - loss: 0.3047 - acc: 0.8763
Epoch 6/100
186/186 [==============================] - 0s 1ms/step - loss: 0.2554 - acc: 0.9194
Epoch 7/100
186/186 [==============================] - 0s 2ms/step - loss: 0.2106 - acc: 0.9301
Epoch 8/100
186/186 [==============================] - 0s 2ms/step - loss: 0.1732 - acc: 0.9677
Epoch 9/100
186/186 [==============================] - 0s 1ms/step - loss: 0.1430 - acc: 0.9785
Epoch 10/100
186/186 [==============================] - 0s 989us/step - loss: 0.1170 - acc: 0.9839
Epoch 11/100
186/186 [======

186/186 [==============================] - 0s 752us/step - loss: 3.1512e-04 - acc: 1.0000
Epoch 84/100
186/186 [==============================] - 0s 624us/step - loss: 3.0248e-04 - acc: 1.0000
Epoch 85/100
186/186 [==============================] - 0s 623us/step - loss: 2.9202e-04 - acc: 1.0000
Epoch 86/100
186/186 [==============================] - 0s 1ms/step - loss: 2.8277e-04 - acc: 1.0000
Epoch 87/100
186/186 [==============================] - 0s 1ms/step - loss: 2.7288e-04 - acc: 1.0000
Epoch 88/100
186/186 [==============================] - 0s 731us/step - loss: 2.6341e-04 - acc: 1.0000
Epoch 89/100
186/186 [==============================] - 0s 645us/step - loss: 2.5397e-04 - acc: 1.0000
Epoch 90/100
186/186 [==============================] - 0s 774us/step - loss: 2.4496e-04 - acc: 1.0000
Epoch 91/100
186/186 [==============================] - 0s 2ms/step - loss: 2.3728e-04 - acc: 1.0000
Epoch 92/100
186/186 [==============================] - 0s 1ms/step - loss: 2.2915e-04 - acc

187/187 [==============================] - 0s 2ms/step - loss: 5.4015e-04 - acc: 1.0000
Epoch 66/100
187/187 [==============================] - 0s 2ms/step - loss: 5.1561e-04 - acc: 1.0000
Epoch 67/100
187/187 [==============================] - 0s 2ms/step - loss: 4.9528e-04 - acc: 1.0000
Epoch 68/100
187/187 [==============================] - 0s 2ms/step - loss: 4.7441e-04 - acc: 1.0000
Epoch 69/100
187/187 [==============================] - 0s 2ms/step - loss: 4.5246e-04 - acc: 1.0000
Epoch 70/100
187/187 [==============================] - 0s 2ms/step - loss: 4.3573e-04 - acc: 1.0000
Epoch 71/100
187/187 [==============================] - 0s 2ms/step - loss: 4.1620e-04 - acc: 1.0000
Epoch 72/100
187/187 [==============================] - 0s 2ms/step - loss: 4.0168e-04 - acc: 1.0000
Epoch 73/100
187/187 [==============================] - 0s 2ms/step - loss: 3.8485e-04 - acc: 1.0000
Epoch 74/100
187/187 [==============================] - 0s 2ms/step - loss: 3.6999e-04 - acc: 1.0000
Epo

187/187 [==============================] - 0s 791us/step - loss: 0.0017 - acc: 1.0000
Epoch 48/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0016 - acc: 1.0000
Epoch 49/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0015 - acc: 1.0000
Epoch 50/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0014 - acc: 1.0000
Epoch 51/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0013 - acc: 1.0000
Epoch 52/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0013 - acc: 1.0000
Epoch 53/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0012 - acc: 1.0000
Epoch 54/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0011 - acc: 1.0000
Epoch 55/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0011 - acc: 1.0000
Epoch 56/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0010 - acc: 1.0000
Epoch 57/100
187/187 [===================

Epoch 29/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0050 - acc: 1.0000
Epoch 30/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0044 - acc: 1.0000
Epoch 31/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0040 - acc: 1.0000
Epoch 32/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0037 - acc: 1.0000
Epoch 33/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0033 - acc: 1.0000
Epoch 34/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0031 - acc: 1.0000
Epoch 35/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0028 - acc: 1.0000
Epoch 36/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0026 - acc: 1.0000
Epoch 37/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0024 - acc: 1.0000
Epoch 38/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0022 - acc: 1.0000
Epoch 39/100
187/187 [========

187/187 [==============================] - 0s 1ms/step - loss: 0.1409 - acc: 0.9626
Epoch 11/100
187/187 [==============================] - 0s 1ms/step - loss: 0.1199 - acc: 0.9786
Epoch 12/100
187/187 [==============================] - 0s 1ms/step - loss: 0.1014 - acc: 1.0000
Epoch 13/100
187/187 [==============================] - 0s 1ms/step - loss: 0.0864 - acc: 0.9947
Epoch 14/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0740 - acc: 0.9947
Epoch 15/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0618 - acc: 1.0000
Epoch 16/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0536 - acc: 1.0000
Epoch 17/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0464 - acc: 1.0000
Epoch 18/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0387 - acc: 1.0000
Epoch 19/100
187/187 [==============================] - 0s 2ms/step - loss: 0.0334 - acc: 1.0000
Epoch 20/100
187/187 [=====================

187/187 [==============================] - 0s 2ms/step - loss: 2.7166e-04 - acc: 1.0000
Epoch 94/100
187/187 [==============================] - 0s 2ms/step - loss: 2.6120e-04 - acc: 1.0000
Epoch 95/100
187/187 [==============================] - 0s 1ms/step - loss: 2.5383e-04 - acc: 1.0000
Epoch 96/100
187/187 [==============================] - 0s 748us/step - loss: 2.4532e-04 - acc: 1.0000
Epoch 97/100
187/187 [==============================] - 0s 599us/step - loss: 2.3866e-04 - acc: 1.0000
Epoch 98/100
187/187 [==============================] - 0s 706us/step - loss: 2.2946e-04 - acc: 1.0000
Epoch 99/100
187/187 [==============================] - 0s 706us/step - loss: 2.2244e-04 - acc: 1.0000
Epoch 100/100
21/21 [==============================] - 2s 107ms/step
Epoch 1/100
187/187 [==============================] - 6s 30ms/step - loss: 0.7433 - acc: 0.4920
Epoch 2/100
187/187 [==============================] - 0s 711us/step - loss: 0.5942 - acc: 0.7112
Epoch 3/100
187/187 [=============

187/187 [==============================] - 0s 2ms/step - loss: 4.5940e-04 - acc: 1.0000
Epoch 76/100
187/187 [==============================] - 0s 2ms/step - loss: 4.4232e-04 - acc: 1.0000
Epoch 77/100
187/187 [==============================] - 0s 2ms/step - loss: 4.2307e-04 - acc: 1.0000
Epoch 78/100
187/187 [==============================] - 0s 2ms/step - loss: 4.0928e-04 - acc: 1.0000
Epoch 79/100
187/187 [==============================] - 0s 2ms/step - loss: 3.9120e-04 - acc: 1.0000
Epoch 80/100
187/187 [==============================] - 0s 2ms/step - loss: 3.7610e-04 - acc: 1.0000
Epoch 81/100
187/187 [==============================] - 0s 2ms/step - loss: 3.6204e-04 - acc: 1.0000
Epoch 82/100
187/187 [==============================] - 0s 2ms/step - loss: 3.5053e-04 - acc: 1.0000
Epoch 83/100
187/187 [==============================] - 0s 2ms/step - loss: 3.3575e-04 - acc: 1.0000
Epoch 84/100
187/187 [==============================] - 0s 2ms/step - loss: 3.2468e-04 - acc: 1.0000
Epo

### 4.3